In [7]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension()
import hvplot.pandas
import matplotlib.pyplot as plt
import datetime as dt

# (0) Importing the Data and Formating the data

In [8]:
df = pd.read_csv("Year.csv",header=0, index_col=False)

df

,Date,Time,System Setting,System Mode,Calendar Event,Program Mode,Cool Set Temp (C),Heat Set Temp (C),Current Temp (C),Current Humidity (%RH),...,tempLoft2,tempMainBedroom (C),tempMainBedroom2,tempLivingRoom (C),tempLivingRoom2,Back window (contact),Main Bedroom window (contact),Living Room window (contact),Office window (contact),Small Bedroom window (contact)
0,2022-01-01,00:00:00,heat,heatOff,NaN,Sleep,26.5,19.5,24.8,39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01,00:05:00,heat,heatOff,NaN,Sleep,26.5,19.5,24.7,39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01,00:10:00,heat,heatOff,NaN,Sleep,26.5,19.5,24.6,39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01,00:15:00,heat,heatOff,NaN,Sleep,26.5,19.5,24.5,39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01,00:20:00,heat,heatOff,NaN,Sleep,26.5,19.5,24.5,39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104827,2022-12-30,23:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104828,2022-12-30,23:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104829,2022-12-30,23:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104830,2022-12-30,23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### We want to change the Date column from string type to datetime type. This is because the slider uses datetime type


In [9]:
type(df.Date[0])

print("The type before was: ", type(df.Date[0]))

df['Date'] = pd.to_datetime(df['Date']).apply(lambda x: x.date())

# Check if it has changed

print("The type now is: ",  type(df.Date[0]))

The type before was:  <class 'str'>
The type now is:  <class 'datetime.date'>


# (1) Just Visualizing the data: Average temperature and humidity per day over the year:

In [12]:
Average_temp = df.groupby('Date', as_index=False, sort=False)['Current Temp (C)'].mean()
Average_humidity = df.groupby('Date', as_index=False, sort=False)['Current Humidity (%RH)'].mean()

# Combine both tables:

result = pd.concat([Average_temp, Average_humidity["Current Humidity (%RH)"]], axis=1, join="inner")

# Looking at the table:

result.head()

,Date,Current Temp (C),Current Humidity (%RH)
0,2022-01-01,24.450000,36.281250
1,2022-01-02,23.303819,22.472222
2,2022-01-03,23.350694,18.142361
3,2022-01-04,23.412847,19.291667
4,2022-01-05,23.069444,21.909722


In [15]:
Temp = result.hvplot(x = 'Date', y = 'Current Temp (C)')
Humidity = result.hvplot(x = 'Date', y = 'Current Humidity (%RH)')

Temp

:Curve   [Date]   (Current Temp (C))

In [16]:
Humidity

:Curve   [Date]   (Current Humidity (%RH))

### Since some of the data is missing we are only going to look at the data from June 29th to Dec 18

# (2) Creating a graph -ndy

In [95]:
# Create the Slider. The slider will slide through each day from June 29th to Dec 18 2022

date_slider = pn.widgets.DateSlider(name='Date:', start=dt.date(2022, 6, 30), end=dt.date(2022, 12, 17), value=dt.date(2022, 6, 30))

# create y_axis button

yaxis_fan = pn.widgets.Select(value='temp or humid', options=['Current Temp (C)', 'Current Humidity (%RH)'], name='Y-axis')



# Function that shows what the slider is doing:

@pn.depends(date_slider.param.value_throttled)
def dat_selected(date):
    return "### Temperature in {}".format(date)

date_slider



DateSlider(end=datetime.date(2022, ..., name='Date:', start=datetime.date(2022, ..., value=datetime.date(2022, ...)

In [96]:
yaxis_fan

Select(name='Y-axis', options=['Current Temp (C)', ...], value='Current Temp (C)')

In [97]:
pn.Row(dat_selected)

Row
    [0] ParamFunction(function)

In [98]:
# Creating a function that when inputing the date will display a plot where the x axis is the time and the y axis is the Temp
#@pn.depends()
@pn.depends(y_axis = yaxis_fan)
@pn.depends(date = date_slider.param.value_throttled)
def plot(date,y_axis):
    one_day_plot = df.loc[df.Date == date]
    
    if y_axis == "Current Temp (C)":
        return one_day_plot.hvplot(x = 'Time', y = 'Current Temp (C)')
    if y_axis == "Current Humidity (%RH)":
        return one_day_plot.hvplot(x = 'Time', y = 'Current Humidity (%RH)')


# An example of the plot 

plot(dt.date(2022, 12, 9),"Current Humidity (%RH)")

:Curve   [Time]   (Current Humidity (%RH))

In [101]:


Columns = pn.Column(dat_selected, date_slider, yaxis_fan)
layout = pn.Row(Columns,pn.bind(plot,date_slider,yaxis_fan))


In [103]:
layout

Row
    [0] Column
        [0] ParamFunction(function)
        [1] DateSlider(end=datetime.date(2022, ..., name='Date:', start=datetime.date(2022, ..., value=datetime.date(2022, ...)
        [2] Select(name='Y-axis', options=['Current Temp (C)', ...], value='Current Temp (C)')
    [1] ParamFunction(function)

### Next step adding when the furnace turns on to the graph and finding possible correlation with gas bill